<a href="https://colab.research.google.com/github/momen-lab/covid19_calender/blob/main/covid19cal_release.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# COVID19 カレンダー 生成器
  
作成者: [@momen_lab](http://www.twitter.com/momen_lab) 
(kenmo34lab@google.com)

## 更新履歴
* 2020-12-04 新規作成
* 2020-12-10 公開

## このノートについて
* 非プログラマーでも簡単な操作でカレンダー画像を生成できるツールを作成した. 
* 動作の概要: 全国／都道府県別の新規陽性者数を [NHK 特設サイト](https://www3.nhk.or.jp/news/special/coronavirus/data/) から取得して以下を行う. 
  * カレンダー化: 曜日ごとの比較を容易にする
  * ヒートマップ化: 視認性を高める

## つかいかた
1. 初期化する (「初期化 (起動後に実行してください)」パネル**下**の [&#x23F5;] ボタンを押す)
  * 30 秒ほどで完了します. 
  * 初期化はノートを開いたときの 1 回だけで構いません (初期化を挟むことなく, 連続してカレンダーを描画できます.) 
  * また, 初期化を行うことでデータソースを NHK サイトから再読み込みできます. 

2. 設定画面を編集する

3. 描画する (「カレンダー描画」下 (設定パネル左) の [&#x23F5;] を押す)

4. カレンダーが表示されます





# 初期化 (起動後に実行してください)

In [ ]:
# 日本語フォントを使用可能に
!pip install japanize-matplotlib
import japanize_matplotlib 

# INSTALL: カラースキーム 
!pip install git+git://github.com/jradavenport/cubehelix.git
import cubehelix

# INSTALL: IPAフォント
!apt-get -y install fonts-ipafont-gothic
# matplotlibのキャッシュをクリア
!rm /root/.cache/matplotlib/fontList.json
!rm /root/.cache/matplotlib/fontlist-v310.json

import pandas as pd
import numpy as np
import requests
import json

import datetime
import locale

import matplotlib.pyplot as plt
import matplotlib
import seaborn as sns

from google.colab import files
from IPython.display import Image

# NHK サイトより json のダウンロード
# j47: 都道府県別
url = 'https://www3.nhk.or.jp/news/special/coronavirus/data/47newpatients-data.json'
resp = requests.get(url)
j47 = resp.json()
# jnationwide: 全国
url = 'https://www3.nhk.or.jp/news/special/coronavirus/data/allpatients-data.json'
resp = requests.get(url)
jnationwide = resp.json()

# フォント設定
font_ja = 'IPAexGothic'
font_an = 'DejaVu Sans'

  Cloning git://github.com/jradavenport/cubehelix.git to /tmp/pip-req-build-7r1fxwif
  Running command git clone -q git://github.com/jradavenport/cubehelix.git /tmp/pip-req-build-7r1fxwif
  Created wheel for cubehelix: filename=cubehelix-0.1.0-cp36-none-any.whl size=4796 sha256=0e4984d61afd3d58944ddc9e382dfd74f6f8416c3d97ceca2f588cc1e593cba0
  Stored in directory: /tmp/pip-ephem-wheel-cache-wyhdzp3o/wheels/6f/c7/13/8d2149e44a6351527d26d45d6ec43d828a10aaffc7cf3af72f
Successfully built cubehelix
Reading package lists... Done
Building dependency tree       
Reading state information... Done
fonts-ipafont-gothic is already the newest version (00303-18ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 14 not upgraded.
rm: cannot remove '/root/.cache/matplotlib/fontList.json': No such file or directory
rm: cannot remove '/root/.cache/matplotlib/fontlist-v310.json': No such file or directory


In [ ]:
def series_formatter(ser, fmt='%Y/%m/%d',
                  begin_dt=datetime.date(2020,3,1), 
                  is_sokuhou=False, 
                  sokuhou_date=None,
                  sokuhou_value=None):
  """
  * 時系列データを加工する
    * 日付インデックスを datetime.date にする
    * 指定開始日を先頭にデータをカットする
    * 速報値がある場合は指定日に指定値を追記する
  """
  # > 日付インデックスを datetime.date 形式に変換
  def str_to_date(s):
    d = datetime.datetime.strptime(s, fmt)
    return datetime.date(d.year, d.month, d.day)
  # >> ただし, 変換済みならなにもしない
  if type(ser.index[0]) is not datetime.date:
    new_index = [str_to_date(s) for s in ser.index]
    ser.index = new_index

  # > 開始日を先頭にカット
  ser = ser[begin_dt.date():]

  # > 速報値がある場合は追加
  if is_sokuhou:
    sokuhou_dt = datetime.datetime.strptime(sokuhou_date, '%Y-%m-%d').date()
  # >> ただし, 速報指定日に既にデータがある場合は上書きしない
    if sokuhou_dt in ser.index:
      is_sokuhou = False
    else:
      ser[sokuhou_dt] = sokuhou_value 

  return ser


def get_youbi(date):
  """
  * 日付に対応する曜日 (日, 月, ..., 土) を取得する. 
  
  Parameters
  ----------
  date: datetime.date
  
  Returns
  ----------
  youbi: str
  """
  week_name_list='月火水木金土日'
  youbi = '%s' % week_name_list[date.weekday()]
  return youbi


def plotter(daily_series):
  """
  * 日別値を読み込みヒートマップを描画する. 

  Parameters
  ----------
  daily_series: pd.Series
    日別値の時系列データ (インデックスは datetime.date)
  
  Returns
  ----------
  fig: matplotlib.figure.Figure
  ax_m: matplotlib.axes.Axes
    主描画領域 (カレンダー)
  ax_s: matplotlib.axes.Axes
    副描画領域 (集計)
  """

  def tabular(ser, fold=7):
    """
    * 日別値を週ごとに折り返してカレンダーをつくる. 

    Parameters
    ----------
    ser: pd.Series
      日別値の時系列データ (インデックスは datetime.date)
    fold: int
      1 行の日数 (通常は 1 週間 = 7 日間)

    Returns
    ----------
    dftab: pd.DataFrame
      表形式のカレンダー
    """
    # > 折り返し
    li = list(ser)
    folded = [li[idx:idx + fold] for idx in range(0,len(li), fold)] 
    dftab = pd.DataFrame(folded)
    
    # > 表側に日付を入れる (ソースの日付を 7 日おきに取得して表側にはめ込む) 
    dftab.index = ser.index[::fold]
    
    # > 表頭に曜日を入れる (曜日を開始日から 7 日分を取得して表頭にはめ込む)
    days = [get_youbi(dftab.index[0] + datetime.timedelta(days=n)) for n in range(fold)]
    dftab.columns = days
    
    return dftab

 
  def tickdate_converter(ticks):
    """
    * 表側日付の表示形式を調整する. 
      * 表側の「年」および「月」は直前と同じ場合は表示しない

    Parameters
    ----------
    ticks: list of text instances

    Returns
    ----------
    ticks_out: list of text instances

    """
    ticks_out = []
    buf_year = ''; buf_month = ''
    for item in ticks:
      i = item.get_text()
      y, m, d = i.split('-')

      if buf_year != y:
          tick = ('/').join([y, m, d])
      elif buf_month != m:
          tick = ('/').join([m, d])
      else:
          tick = d
      buf_year, buf_month = y, m

      item.set_text(tick)
      ticks_out += [item]
      
    return ticks_out


  # > 表形式のカレンダーを生成
  dftab = tabular(daily_series)

  # > カラーバーの最大値を計算
  nmax = dftab.max().max()
  divider = 5 * 10 ** (np.ceil(np.log10(nmax))-2)
  vmax = np.ceil(nmax / divider) * divider

  # > 合計テーブルの作成 (平均の計算は合計(sum)÷日数(days))
  dfsum = pd.DataFrame(dftab.sum(axis=1), columns=['sum'])
  dfsum['days'] =dftab.apply(lambda x: x.count(), axis=1)
  dfsum['avg'] = dfsum['sum'] / dfsum['days']

  # > ヒートマップの描画
  # >> 描画オブジェクト生成
  fig, (ax_m, ax_s) = plt.subplots(nrows=1, ncols=2, sharey=True, dpi=72,
                                  gridspec_kw={ 'width_ratios': [6.8, 1]},
                                  figsize=(6,15), facecolor='w')
  # 週計が 7 桁以上になるなら width_ratios は要修正
  plt.subplots_adjust(wspace=0.02) # 横ヒゲの長さくらい

  # >> 日別ヒートマップ描画
  heatmap_daily = sns.heatmap(dftab, annot=True, fmt='.0f', cmap=cmap, vmin=0, vmax=vmax, linewidths=0,
                              cbar=False, annot_kws={'family': font_an}, ax=ax_m)
  # >> 集計ヒートマップ描画
  heatmap_weekly = sns.heatmap(pd.DataFrame(dfsum['avg']), annot=True, fmt=' >6.1f', cmap=cmap, vmin=0, vmax=vmax,
                              cbar=False, annot_kws={'ha': 'right', 'family': font_an}, ax=ax_s)

  # > 文字の書式設定
  # >> 日別テーブル書式設定
  ax_m.set_yticklabels(tickdate_converter(ax_m.get_yticklabels()), fontname = font_an)      
  ax_m.set_xticklabels(dftab.columns, fontname = font_ja, fontsize=12)
  
  # >> 週計テーブル書式設定
  ax_s.set_xticklabels(labels=['週計'], fontname = font_ja, fontsize=12)

  for idx, t in enumerate(ax_s.texts):
    trans = t.get_transform()
    offs = matplotlib.transforms.ScaledTranslation(0.37,0,
                    matplotlib.transforms.IdentityTransform())
    t.set_transform( offs + trans )
    txt = t.get_text()
    t.set_text(int(dfsum['sum'].iloc[idx]))

  # > カラーバーの描画
  axpos = ax_s.get_position()
  ax_cbar = fig.add_axes([0.93, axpos.y0+0.2, 0.05, axpos.height-0.4])
  norm = matplotlib.colors.Normalize(vmin=0, vmax=vmax)
  mappable = matplotlib.cm.ScalarMappable(cmap=cmap,norm=norm)
  mappable._A = []
  cb = fig.colorbar(mappable, cax=ax_cbar)
  cb.outline.set_visible(False)

  # > タイトルおよび注釈の書き込み
  fd={'family': font_ja}

  ax_m.set_title('COVID-19 新規感染者の報告数（'+pref+'）', fontdict={'family': font_ja, 'size':14})
  s = '※週計の着色は, 週計÷日数 (週平均) の値に基づく.\n'
  if is_sokuhou == True:
    s = s + '※'+sokuhou_date+'は速報値.\n'
  s = s + 'データ取得元: NHK' +\
      ', 描画ツール提供:（ヽ´ん`）(@momen_lab)'

  ax_m.text(0,len(dftab)+1.2, s, ha='left', fontdict=fd, va='top')

  return fig, (ax_m, ax_s)

# カレンダー描画

In [ ]:
#@markdown ← 描画ボタン
#@markdown ### 設定パネル
#today = datetime.date.today().strftime('%Y-%m-%d')

#@markdown 描画対象の都道府県
pref = '\u5168\u56FD' #@param ['全国', '北海道', '青森県', '岩手県', '宮城県', '秋田県', '山形県', '福島県', '茨城県', '栃木県', '群馬県', '埼玉県', '千葉県', '東京都', '神奈川県', '新潟県', '富山県', '石川県', '福井県', '山梨県', '長野県', '岐阜県', '静岡県', '愛知県', '三重県', '滋賀県', '京都府', '大阪府', '兵庫県', '奈良県', '和歌山県', '鳥取県', '島根県', '岡山県', '広島県', '山口県', '徳島県', '香川県', '愛媛県', '高知県', '福岡県', '佐賀県', '長崎県', '熊本県', '大分県', '宮崎県', '鹿児島県', '沖縄県']

#@markdown 開始日
begin_date = '2020-03-01' #@param {type:"date"}
begin_dt = datetime.datetime.strptime(begin_date, '%Y-%m-%d')

#@markdown 速報値（ある場合はチェックを入れて, 日付と速報値を記入する）
is_sokuhou = False #@param {type:"boolean"}
sokuhou_date = '2020-12-10' #@param {type:"date"}
sokuhou_value = 70#@param {type:"integer"}

#@markdown 自動保存
is_autosave = False #@param {type:"boolean"}

#@markdown カラーパレット
which_cmap = 'Light' #@param ['Light', 'Dark']

# カラースキーム設定
cmap_dic = {
    'Dark': cubehelix.cmap(start=100, rot=-1.4),
    'Light': sns.cubehelix_palette(start=1.7, rot=1.5, dark=0.3, light=0.95, as_cmap=True)
}
cmap = cmap_dic[which_cmap]

# プロット対象の時系列を取得
if pref == '全国': 
  df = pd.json_normalize(jnationwide, 'dataAll')#.set_index(['name'])
  ser = pd.Series(data=df.loc[0,'data'], index=jnationwide['category'], name='全国／１日ごとの発表数')

else:
  df = pd.json_normalize(j47, 'data47').set_index(['name'])
  ser = pd.Series(data=df.loc[pref, 'data'], index=j47['category'], name=pref+'／１日ごとの発表数')

fig, (ax_m, ax_s) = plotter(series_formatter(ser, 
                          begin_dt=begin_dt, 
                          is_sokuhou=is_sokuhou, 
                          sokuhou_date=sokuhou_date, 
                          sokuhou_value=sokuhou_value))

if is_autosave:
  out_filename = './'+pref+'_'+ser.index[-1].strftime('%Y-%m-%d')+'.png'
  plt.savefig(out_filename, bbox_inches='tight')
  files.download(out_filename) 